In [1]:
!pwd

In [2]:
# Configure env variables

# TODO: change path
import os
os.environ["DARM_MUJOCO_PATH"] = "/home/daniel/DARM/darm_mujoco"
os.getenv('DARM_MUJOCO_PATH')

'/home/daniel/DARM/darm_mujoco'

In [3]:
import ray
from ray.rllib.algorithms.es import ESConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym
from darm_gym_env import DARMSFEnv

In [4]:
# env_creator = lambda env_config: gym.make("darm/DarmSFHand-v0", render_mode=None, hand_name="hand1") # DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1") # 

def make_env(env_config):
    env = gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)
    return env
env_creator = lambda env_config: make_env(env_config) #gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)

register_env("darm/DarmSFHand-v0", env_creator)

In [5]:
# TODO:
# change: rollout_workers
# change: gpu

config = (
    ESConfig()
    .environment(
        env="darm/DarmSFHand-v0"
    )
    .rollouts(
        num_rollout_workers=3,
        num_envs_per_worker=8,
        rollout_fragment_length=1,
        recreate_failed_workers=True,
        num_consecutive_worker_failures_tolerance=10,
        restart_failed_sub_environments=True,
    )
    .resources(num_gpus=0)
    # .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .framework(framework="torch")
)
config.to_dict()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'darm/DarmSFHand-v0',
 'env_config': {},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 'num_envs_per_worker': 8,
 'sample_collector': ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector,
 's

In [6]:
# TODO:
# change: rollout_workers
# change: gpu
# change: tags
# change: name

wandb_init = dict(
    save_code=True,
    resume=True,
    config={
        "env": "DARMSFHand-v0",
        "num_rollout_workers": 3,
        "num_envs_per_worker": 8,
        "recreate_failed_workers": True,
        "num_consecutive_worker_failures_tolerance": 10,
        "restart_failed_sub_environments": True,
        "num_gpus": 0,
        "framework": "torch"
    },
    tags=["single_finger", "es", "delta_target", "test", "local", "no_vel_penalty", "effort_penalty"],
    notes="Fixed the env to use targets that are delta increaments from the starting state. Removed velocity penalty, and used only effort penalty",
    name="ES_Test_DARMSF_DELTA_TARGET"
    # job_type=
    # monitor_gym=
)

In [7]:
%cd /home/daniel/DARM/darm-mujoco/darm_training/

In [8]:
# TODO: 
# change: name
# change: checkpoint_freq

sync_config = tune.SyncConfig()

tuner = tune.Tuner(
    "ES",
    run_config=air.RunConfig(
        name="ES_Test_DARMSF_DELTA_TARGET",
        local_dir=f"{os.getenv('DARM_MUJOCO_PATH')}/darm_training/results",
        sync_config=sync_config,
        stop={"training_iteration": 10_000, "episode_reward_mean": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_score_attribute="episode_reward_mean",  # or leave to save last chkpts
            checkpoint_score_order="max",
            checkpoint_frequency=1,  #50,
            num_to_keep=3
        ),
        callbacks=[
                WandbLoggerCallback(project="DARM", 
                                    api_key="392c8a47eb0658eb5c71190757a69110e2140f4a",
                                    save_checkpoints=True, 
                                    **wandb_init)
            ],
        ),
    param_space=config.to_dict()
)

results = tuner.fit()